In [1]:
import torch
import numpy
import pandas as pd

In [2]:
from torch.utils.data import TensorDataset, DataLoader

In [3]:
df = pd.read_csv('housing.csv', delim_whitespace=True)

In [4]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [110]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [173]:
df_X = df.drop(columns=['MEDV'])
df_y = df['MEDV']
min_y = df_y.min()
max_y = df_y.max() 
df_X = (df_X - df_X.min()) / df_X.max()
df_y = (df_y - min_y) / max_y


In [174]:
X_train, y_train = torch.Tensor(df_X.to_numpy()[:400]), torch.Tensor(df_y.to_numpy()[:400])
X_test, y_test = torch.Tensor(df_X.to_numpy()[400:]), torch.Tensor(df_y.to_numpy()[400:])


In [175]:
X_test.shape

torch.Size([106, 13])

In [176]:
dataset = TensorDataset(X_train, y_train)

In [177]:
dataset

In [178]:
batch_size = 10
data_iter = DataLoader(dataset, batch_size, shuffle=True)

In [184]:
model = torch.nn.Sequential(torch.nn.Linear(13, 1))

In [185]:
for X, y in data_iter:
    print(model.forward(X).reshape(-1))
    break

tensor([-0.2048, -0.0961,  0.1261, -0.0302,  0.2169, -0.3231,  0.0163, -0.0340,
         0.3301,  0.0723], grad_fn=<AsStridedBackward>)


In [186]:
loss = torch.nn.MSELoss(reduction='mean')

In [187]:
model.parameters

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=13, out_features=1, bias=True)
)>

In [188]:
trainer = torch.optim.SGD(model.parameters(), lr=0.001)

In [189]:
num_epochs = 2000
for epoch in range(num_epochs):
    for X, y in data_iter:
        trainer.zero_grad()
        l = loss(model.forward(X).reshape(-1), y)
        l.backward()
        trainer.step()
    l = loss(model.forward(X_test).reshape(-1), y_test)   
    print('epoch %d, loss: %f' % (epoch, l.item() * max_y + min_y))
                         

epoch 0, loss: 8.017367
epoch 1, loss: 9.062236
epoch 2, loss: 10.107057
epoch 3, loss: 11.023247
epoch 4, loss: 11.800358
epoch 5, loss: 12.428633
epoch 6, loss: 12.906932
epoch 7, loss: 13.242384
epoch 8, loss: 13.481633
epoch 9, loss: 13.604192
epoch 10, loss: 13.676477
epoch 11, loss: 13.704764
epoch 12, loss: 13.676066
epoch 13, loss: 13.604573
epoch 14, loss: 13.505403
epoch 15, loss: 13.387401
epoch 16, loss: 13.246628
epoch 17, loss: 13.102827
epoch 18, loss: 12.955404
epoch 19, loss: 12.792199
epoch 20, loss: 12.643680
epoch 21, loss: 12.478216
epoch 22, loss: 12.304049
epoch 23, loss: 12.151262
epoch 24, loss: 11.985997
epoch 25, loss: 11.821342
epoch 26, loss: 11.658803
epoch 27, loss: 11.512671
epoch 28, loss: 11.368106
epoch 29, loss: 11.229560
epoch 30, loss: 11.095310
epoch 31, loss: 10.941547
epoch 32, loss: 10.809201
epoch 33, loss: 10.679872
epoch 34, loss: 10.549720
epoch 35, loss: 10.430964
epoch 36, loss: 10.302201
epoch 37, loss: 10.180968
epoch 38, loss: 10.07300

epoch 325, loss: 5.457615
epoch 326, loss: 5.457257
epoch 327, loss: 5.456569
epoch 328, loss: 5.456699
epoch 329, loss: 5.455434
epoch 330, loss: 5.454729
epoch 331, loss: 5.454528
epoch 332, loss: 5.454643
epoch 333, loss: 5.454475
epoch 334, loss: 5.453693
epoch 335, loss: 5.453795
epoch 336, loss: 5.452662
epoch 337, loss: 5.452366
epoch 338, loss: 5.451968
epoch 339, loss: 5.450972
epoch 340, loss: 5.451254
epoch 341, loss: 5.451353
epoch 342, loss: 5.450386
epoch 343, loss: 5.450800
epoch 344, loss: 5.451279
epoch 345, loss: 5.451543
epoch 346, loss: 5.451149
epoch 347, loss: 5.450685
epoch 348, loss: 5.450283
epoch 349, loss: 5.449374
epoch 350, loss: 5.449259
epoch 351, loss: 5.449017
epoch 352, loss: 5.448696
epoch 353, loss: 5.448079
epoch 354, loss: 5.448109
epoch 355, loss: 5.447338
epoch 356, loss: 5.447398
epoch 357, loss: 5.447329
epoch 358, loss: 5.446691
epoch 359, loss: 5.446096
epoch 360, loss: 5.445790
epoch 361, loss: 5.446505
epoch 362, loss: 5.446977
epoch 363, l

epoch 649, loss: 5.454041
epoch 650, loss: 5.454062
epoch 651, loss: 5.454316
epoch 652, loss: 5.454342
epoch 653, loss: 5.453931
epoch 654, loss: 5.453745
epoch 655, loss: 5.454152
epoch 656, loss: 5.454277
epoch 657, loss: 5.454496
epoch 658, loss: 5.454837
epoch 659, loss: 5.454816
epoch 660, loss: 5.455275
epoch 661, loss: 5.455696
epoch 662, loss: 5.455745
epoch 663, loss: 5.455839
epoch 664, loss: 5.456017
epoch 665, loss: 5.456164
epoch 666, loss: 5.456340
epoch 667, loss: 5.456339
epoch 668, loss: 5.456495
epoch 669, loss: 5.456699
epoch 670, loss: 5.456917
epoch 671, loss: 5.456411
epoch 672, loss: 5.456650
epoch 673, loss: 5.456236
epoch 674, loss: 5.456551
epoch 675, loss: 5.456958
epoch 676, loss: 5.457011
epoch 677, loss: 5.456943
epoch 678, loss: 5.456996
epoch 679, loss: 5.457323
epoch 680, loss: 5.457567
epoch 681, loss: 5.457873
epoch 682, loss: 5.458134
epoch 683, loss: 5.457875
epoch 684, loss: 5.458275
epoch 685, loss: 5.458346
epoch 686, loss: 5.458639
epoch 687, l

epoch 977, loss: 5.492951
epoch 978, loss: 5.492860
epoch 979, loss: 5.493035
epoch 980, loss: 5.493256
epoch 981, loss: 5.493273
epoch 982, loss: 5.493479
epoch 983, loss: 5.494018
epoch 984, loss: 5.494091
epoch 985, loss: 5.494010
epoch 986, loss: 5.494318
epoch 987, loss: 5.494179
epoch 988, loss: 5.494385
epoch 989, loss: 5.494667
epoch 990, loss: 5.494462
epoch 991, loss: 5.494674
epoch 992, loss: 5.494720
epoch 993, loss: 5.494645
epoch 994, loss: 5.494814
epoch 995, loss: 5.494851
epoch 996, loss: 5.495061
epoch 997, loss: 5.495024
epoch 998, loss: 5.495271
epoch 999, loss: 5.495537
epoch 1000, loss: 5.495839
epoch 1001, loss: 5.495931
epoch 1002, loss: 5.496202
epoch 1003, loss: 5.496376
epoch 1004, loss: 5.496329
epoch 1005, loss: 5.496459
epoch 1006, loss: 5.495805
epoch 1007, loss: 5.496195
epoch 1008, loss: 5.496001
epoch 1009, loss: 5.496156
epoch 1010, loss: 5.496367
epoch 1011, loss: 5.496121
epoch 1012, loss: 5.496346
epoch 1013, loss: 5.496615
epoch 1014, loss: 5.4966

epoch 1303, loss: 5.518464
epoch 1304, loss: 5.518620
epoch 1305, loss: 5.518381
epoch 1306, loss: 5.518462
epoch 1307, loss: 5.518315
epoch 1308, loss: 5.518498
epoch 1309, loss: 5.518984
epoch 1310, loss: 5.519210
epoch 1311, loss: 5.519024
epoch 1312, loss: 5.519561
epoch 1313, loss: 5.519587
epoch 1314, loss: 5.519498
epoch 1315, loss: 5.519667
epoch 1316, loss: 5.519745
epoch 1317, loss: 5.519664
epoch 1318, loss: 5.519935
epoch 1319, loss: 5.520176
epoch 1320, loss: 5.520138
epoch 1321, loss: 5.520088
epoch 1322, loss: 5.519890
epoch 1323, loss: 5.519458
epoch 1324, loss: 5.519306
epoch 1325, loss: 5.519338
epoch 1326, loss: 5.519531
epoch 1327, loss: 5.519441
epoch 1328, loss: 5.519189
epoch 1329, loss: 5.519097
epoch 1330, loss: 5.519530
epoch 1331, loss: 5.519448
epoch 1332, loss: 5.519787
epoch 1333, loss: 5.519955
epoch 1334, loss: 5.519953
epoch 1335, loss: 5.520052
epoch 1336, loss: 5.520519
epoch 1337, loss: 5.520335
epoch 1338, loss: 5.520381
epoch 1339, loss: 5.520338
e

epoch 1631, loss: 5.530604
epoch 1632, loss: 5.530850
epoch 1633, loss: 5.530541
epoch 1634, loss: 5.530916
epoch 1635, loss: 5.530942
epoch 1636, loss: 5.531328
epoch 1637, loss: 5.531549
epoch 1638, loss: 5.531451
epoch 1639, loss: 5.531619
epoch 1640, loss: 5.531386
epoch 1641, loss: 5.531696
epoch 1642, loss: 5.531711
epoch 1643, loss: 5.531745
epoch 1644, loss: 5.531807
epoch 1645, loss: 5.532045
epoch 1646, loss: 5.531570
epoch 1647, loss: 5.531790
epoch 1648, loss: 5.531624
epoch 1649, loss: 5.531560
epoch 1650, loss: 5.531449
epoch 1651, loss: 5.531730
epoch 1652, loss: 5.531721
epoch 1653, loss: 5.531663
epoch 1654, loss: 5.531747
epoch 1655, loss: 5.531924
epoch 1656, loss: 5.532121
epoch 1657, loss: 5.532023
epoch 1658, loss: 5.531903
epoch 1659, loss: 5.531812
epoch 1660, loss: 5.531788
epoch 1661, loss: 5.531911
epoch 1662, loss: 5.532118
epoch 1663, loss: 5.532250
epoch 1664, loss: 5.532331
epoch 1665, loss: 5.532269
epoch 1666, loss: 5.532390
epoch 1667, loss: 5.532382
e

epoch 1935, loss: 5.535941
epoch 1936, loss: 5.535637
epoch 1937, loss: 5.536030
epoch 1938, loss: 5.535776
epoch 1939, loss: 5.535982
epoch 1940, loss: 5.535696
epoch 1941, loss: 5.536142
epoch 1942, loss: 5.536255
epoch 1943, loss: 5.536163
epoch 1944, loss: 5.536330
epoch 1945, loss: 5.536424
epoch 1946, loss: 5.536260
epoch 1947, loss: 5.536538
epoch 1948, loss: 5.536673
epoch 1949, loss: 5.536738
epoch 1950, loss: 5.536935
epoch 1951, loss: 5.537242
epoch 1952, loss: 5.536931
epoch 1953, loss: 5.536444
epoch 1954, loss: 5.536510
epoch 1955, loss: 5.536337
epoch 1956, loss: 5.536508
epoch 1957, loss: 5.536718
epoch 1958, loss: 5.536876
epoch 1959, loss: 5.536618
epoch 1960, loss: 5.536595
epoch 1961, loss: 5.536462
epoch 1962, loss: 5.536417
epoch 1963, loss: 5.536414
epoch 1964, loss: 5.536024
epoch 1965, loss: 5.536260
epoch 1966, loss: 5.536485
epoch 1967, loss: 5.536686
epoch 1968, loss: 5.536199
epoch 1969, loss: 5.536286
epoch 1970, loss: 5.536131
epoch 1971, loss: 5.536035
e

In [190]:
print('epoch %d, loss: %f' % (epoch, l.item() * max_y + min_y))


epoch 1999, loss: 5.536666
